# 15 - MLflow + LangGraph: Tracking de Workflows de AI

## 🎯 Objetivos
- Integrar MLflow con LangGraph para tracking de flujos LLM
- Crear workflows de AI complejos con grafos de estados
- Trackear prompts, respuestas y métricas
- Comparar diferentes configuraciones de LLM
- Versionado de prompts y chains
- Evaluación de calidad de respuestas

## 📚 Tecnologías
- **MLflow**: Experiment tracking y model registry
- **LangGraph**: Framework para workflows de LLMs
- **LangChain**: Herramientas para LLMs
- **OpenAI/Anthropic**: APIs de LLMs (opcional)

## ⭐ Complejidad: Avanzado

## 1. Instalación y Setup

In [ ]:
# Instalar dependencias
!pip install mlflow langgraph langchain langchain-core langchain-community langchain-openai pandas numpy matplotlib -q

In [ ]:
import mlflow
import mlflow.langchain
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import datetime
from pathlib import Path
import json
import warnings
warnings.filterwarnings('ignore')

# LangGraph y LangChain
from langgraph.graph import Graph, StateGraph, END
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser, JsonOutputParser
from typing import TypedDict, Annotated, Sequence
import operator

print(f"✅ MLflow version: {mlflow.__version__}")
print(f"✅ Imports completados")
print(f"\n💡 Nota: Este notebook usa LLMs simulados para demo")
print(f"   Para usar LLMs reales, configura API keys de OpenAI o Anthropic")

## 2. Configurar MLflow

In [ ]:
# Configurar MLflow
mlflow.set_tracking_uri("./mlruns")
experiment_name = "langgraph_workflows"
mlflow.set_experiment(experiment_name)

print(f"✅ MLflow configurado")
print(f"📊 Experimento: {experiment_name}")
print(f"📁 Tracking URI: {mlflow.get_tracking_uri()}")
print(f"\n💡 Para ver la UI ejecuta: mlflow ui --port 5000")

## 3. LLM Simulado para Demo

Usaremos un LLM simulado para demostración. En producción, usa OpenAI, Anthropic, etc.

In [ ]:
class MockLLM:
    """
    LLM simulado para propósitos de demostración.
    En producción, reemplaza con:
    - ChatOpenAI de langchain_openai
    - ChatAnthropic de langchain_anthropic
    - Otros providers
    """
    
    def __init__(self, model_name="mock-gpt-4", temperature=0.7):
        self.model_name = model_name
        self.temperature = temperature
        self.call_count = 0
        
    def invoke(self, messages):
        """Simula una llamada a LLM"""
        self.call_count += 1
        
        # Simular diferentes respuestas según el contexto
        last_message = messages[-1] if isinstance(messages, list) else str(messages)
        
        if isinstance(last_message, HumanMessage):
            content = last_message.content
        else:
            content = str(last_message)
        
        # Respuestas simuladas
        if "resume" in content.lower() or "summarize" in content.lower():
            response = "Resumen: El texto habla sobre machine learning y sus aplicaciones en la industria moderna."
        elif "translate" in content.lower() or "traduce" in content.lower():
            response = "Translation: This is a simulated translation of the input text."
        elif "analiza" in content.lower() or "analyze" in content.lower():
            response = "Análisis: Los datos muestran una tendencia positiva con crecimiento del 15% anual."
        elif "classify" in content.lower() or "clasifica" in content.lower():
            response = '{"category": "Technology", "sentiment": "positive", "confidence": 0.85}'
        else:
            response = f"Respuesta simulada para: {content[:50]}... (llamada #{self.call_count})"
        
        return AIMessage(content=response)
    
    def __repr__(self):
        return f"MockLLM(model={self.model_name}, temp={self.temperature})"

# Crear instancia
llm = MockLLM(model_name="mock-gpt-4", temperature=0.7)
print(f"✅ LLM simulado creado: {llm}")
print(f"\n💡 Para usar LLM real:")
print(f"   from langchain_openai import ChatOpenAI")
print(f"   llm = ChatOpenAI(model='gpt-4', temperature=0.7)")

## 4. Workflow Simple con LangGraph y MLflow

In [ ]:
# Definir estado del grafo
class SimpleState(TypedDict):
    messages: Sequence[HumanMessage | AIMessage]
    current_step: str
    metadata: dict

def create_simple_workflow():
    """
    Crea un workflow simple de procesamiento de texto
    """
    
    # Nodos del grafo
    def analyze_input(state: SimpleState):
        """Analiza el input del usuario"""
        print("🔍 Analizando input...")
        messages = state["messages"]
        last_message = messages[-1]
        
        # Simular análisis
        word_count = len(last_message.content.split())
        
        return {
            "messages": messages,
            "current_step": "analyzed",
            "metadata": {"word_count": word_count, "language": "es"}
        }
    
    def process_with_llm(state: SimpleState):
        """Procesa con LLM"""
        print("🤖 Procesando con LLM...")
        messages = state["messages"]
        
        # Llamar al LLM
        response = llm.invoke(messages)
        messages.append(response)
        
        return {
            "messages": messages,
            "current_step": "processed",
            "metadata": state["metadata"]
        }
    
    def format_output(state: SimpleState):
        """Formatea la salida"""
        print("📝 Formateando output...")
        messages = state["messages"]
        last_response = messages[-1].content
        
        formatted = f"**Respuesta del Asistente:**\n{last_response}"
        messages.append(AIMessage(content=formatted))
        
        return {
            "messages": messages,
            "current_step": "completed",
            "metadata": state["metadata"]
        }
    
    # Crear grafo
    workflow = StateGraph(SimpleState)
    
    # Agregar nodos
    workflow.add_node("analyze", analyze_input)
    workflow.add_node("process", process_with_llm)
    workflow.add_node("format", format_output)
    
    # Definir flujo
    workflow.set_entry_point("analyze")
    workflow.add_edge("analyze", "process")
    workflow.add_edge("process", "format")
    workflow.add_edge("format", END)
    
    return workflow.compile()

# Crear workflow
simple_workflow = create_simple_workflow()
print("✅ Workflow simple creado")

## 5. Ejecutar y Trackear Workflow con MLflow

In [ ]:
def run_and_track_workflow(workflow, user_input, run_name):
    """
    Ejecuta workflow y trackea con MLflow
    """
    with mlflow.start_run(run_name=run_name):
        
        # Log parámetros
        mlflow.log_param("workflow_type", "simple")
        mlflow.log_param("llm_model", llm.model_name)
        mlflow.log_param("llm_temperature", llm.temperature)
        mlflow.log_param("user_input", user_input)
        
        # Estado inicial
        initial_state = {
            "messages": [HumanMessage(content=user_input)],
            "current_step": "start",
            "metadata": {}
        }
        
        # Ejecutar workflow
        start_time = datetime.now()
        result = workflow.invoke(initial_state)
        end_time = datetime.now()
        
        execution_time = (end_time - start_time).total_seconds()
        
        # Log métricas
        mlflow.log_metric("execution_time_seconds", execution_time)
        mlflow.log_metric("total_messages", len(result["messages"]))
        mlflow.log_metric("word_count", result["metadata"].get("word_count", 0))
        mlflow.log_metric("llm_calls", llm.call_count)
        
        # Guardar conversación
        conversation = []
        for msg in result["messages"]:
            conversation.append({
                "type": type(msg).__name__,
                "content": msg.content
            })
        
        with open('conversation.json', 'w') as f:
            json.dump(conversation, f, indent=2)
        mlflow.log_artifact('conversation.json')
        
        # Log metadata
        mlflow.log_dict(result["metadata"], "metadata.json")
        
        # Tags
        mlflow.set_tag("workflow_status", result["current_step"])
        mlflow.set_tag("language", result["metadata"].get("language", "unknown"))
        
        print(f"\n✅ Workflow completado en {execution_time:.2f}s")
        print(f"📊 Total mensajes: {len(result['messages'])}")
        print(f"📊 Llamadas LLM: {llm.call_count}")
        print(f"\n📝 Respuesta final:")
        print(result["messages"][-1].content)
        
        return result

# Ejecutar múltiples variaciones
test_inputs = [
    "Resume este documento sobre machine learning y sus aplicaciones.",
    "Analiza las tendencias de ventas del último trimestre.",
    "Clasifica el siguiente texto en categorías: Tecnología innovadora para el futuro."
]

results = []
for i, user_input in enumerate(test_inputs, 1):
    print(f"\n{'='*60}")
    print(f"Ejecutando workflow {i}/{len(test_inputs)}")
    print(f"{'='*60}")
    
    # Reset LLM call count
    llm.call_count = 0
    
    result = run_and_track_workflow(
        simple_workflow,
        user_input,
        f"simple_workflow_test_{i}"
    )
    results.append(result)

## 6. Workflow Avanzado: Multi-Agente con Decisiones

In [ ]:
# Estado para workflow multi-agente
class AgentState(TypedDict):
    messages: Sequence[HumanMessage | AIMessage]
    next_agent: str
    task_type: str
    results: dict

def create_multi_agent_workflow():
    """
    Workflow con múltiples agentes especializados
    """
    
    def router(state: AgentState):
        """Decide qué agente debe procesar la tarea"""
        print("🔀 Router: Analizando tipo de tarea...")
        
        last_message = state["messages"][-1].content.lower()
        
        if "resume" in last_message or "summarize" in last_message:
            task_type = "summarization"
            next_agent = "summarizer"
        elif "translate" in last_message or "traduce" in last_message:
            task_type = "translation"
            next_agent = "translator"
        elif "analiza" in last_message or "analyze" in last_message:
            task_type = "analysis"
            next_agent = "analyzer"
        else:
            task_type = "general"
            next_agent = "general_agent"
        
        print(f"   → Tipo de tarea: {task_type}")
        print(f"   → Agente asignado: {next_agent}")
        
        return {
            "messages": state["messages"],
            "next_agent": next_agent,
            "task_type": task_type,
            "results": {}
        }
    
    def summarizer_agent(state: AgentState):
        """Agente especializado en resúmenes"""
        print("📝 Summarizer Agent trabajando...")
        
        prompt = f"Resume el siguiente texto de manera concisa: {state['messages'][-1].content}"
        response = llm.invoke([HumanMessage(content=prompt)])
        
        state["messages"].append(response)
        state["results"]["summary"] = response.content
        
        return state
    
    def analyzer_agent(state: AgentState):
        """Agente especializado en análisis"""
        print("📊 Analyzer Agent trabajando...")
        
        prompt = f"Analiza el siguiente contenido: {state['messages'][-1].content}"
        response = llm.invoke([HumanMessage(content=prompt)])
        
        state["messages"].append(response)
        state["results"]["analysis"] = response.content
        
        return state
    
    def translator_agent(state: AgentState):
        """Agente especializado en traducción"""
        print("🌍 Translator Agent trabajando...")
        
        prompt = f"Traduce el siguiente texto: {state['messages'][-1].content}"
        response = llm.invoke([HumanMessage(content=prompt)])
        
        state["messages"].append(response)
        state["results"]["translation"] = response.content
        
        return state
    
    def general_agent(state: AgentState):
        """Agente general"""
        print("🤖 General Agent trabajando...")
        
        response = llm.invoke(state["messages"])
        state["messages"].append(response)
        state["results"]["response"] = response.content
        
        return state
    
    def should_continue(state: AgentState) -> str:
        """Decide si continuar o terminar"""
        return state["next_agent"]
    
    # Crear grafo
    workflow = StateGraph(AgentState)
    
    # Agregar nodos
    workflow.add_node("router", router)
    workflow.add_node("summarizer", summarizer_agent)
    workflow.add_node("analyzer", analyzer_agent)
    workflow.add_node("translator", translator_agent)
    workflow.add_node("general_agent", general_agent)
    
    # Definir flujo con decisiones
    workflow.set_entry_point("router")
    
    workflow.add_conditional_edges(
        "router",
        should_continue,
        {
            "summarizer": "summarizer",
            "analyzer": "analyzer",
            "translator": "translator",
            "general_agent": "general_agent"
        }
    )
    
    # Todos los agentes terminan
    workflow.add_edge("summarizer", END)
    workflow.add_edge("analyzer", END)
    workflow.add_edge("translator", END)
    workflow.add_edge("general_agent", END)
    
    return workflow.compile()

# Crear workflow multi-agente
multi_agent_workflow = create_multi_agent_workflow()
print("✅ Workflow multi-agente creado")

## 7. Ejecutar Workflow Multi-Agente con MLflow

In [ ]:
def run_multi_agent_workflow(workflow, user_input, run_name):
    """
    Ejecuta workflow multi-agente y trackea con MLflow
    """
    with mlflow.start_run(run_name=run_name):
        
        # Log parámetros
        mlflow.log_param("workflow_type", "multi_agent")
        mlflow.log_param("llm_model", llm.model_name)
        mlflow.log_param("user_input", user_input)
        
        # Estado inicial
        initial_state = {
            "messages": [HumanMessage(content=user_input)],
            "next_agent": "",
            "task_type": "",
            "results": {}
        }
        
        # Ejecutar
        llm.call_count = 0
        start_time = datetime.now()
        result = workflow.invoke(initial_state)
        end_time = datetime.now()
        
        execution_time = (end_time - start_time).total_seconds()
        
        # Log métricas
        mlflow.log_metric("execution_time_seconds", execution_time)
        mlflow.log_metric("total_messages", len(result["messages"]))
        mlflow.log_metric("llm_calls", llm.call_count)
        
        # Log parámetros de routing
        mlflow.log_param("task_type", result["task_type"])
        mlflow.log_param("selected_agent", result["next_agent"])
        
        # Guardar resultados
        with open('agent_results.json', 'w') as f:
            json.dump(result["results"], f, indent=2)
        mlflow.log_artifact('agent_results.json')
        
        # Guardar conversación completa
        conversation = []
        for msg in result["messages"]:
            conversation.append({
                "type": type(msg).__name__,
                "content": msg.content
            })
        
        with open('multi_agent_conversation.json', 'w') as f:
            json.dump(conversation, f, indent=2)
        mlflow.log_artifact('multi_agent_conversation.json')
        
        # Tags
        mlflow.set_tag("agent_used", result["next_agent"])
        mlflow.set_tag("task_type", result["task_type"])
        
        print(f"\n✅ Workflow multi-agente completado en {execution_time:.2f}s")
        print(f"📊 Agente usado: {result['next_agent']}")
        print(f"📊 Tipo de tarea: {result['task_type']}")
        print(f"📊 Llamadas LLM: {llm.call_count}")
        print(f"\n📝 Resultado:")
        print(result["messages"][-1].content)
        
        return result

# Ejecutar con diferentes tipos de tareas
test_tasks = [
    "Resume este artículo sobre inteligencia artificial en medicina.",
    "Analiza las métricas de rendimiento del último sprint.",
    "Traduce este texto al inglés: Hola, ¿cómo estás?",
    "¿Cuál es la capital de Francia?"  # General
]

multi_agent_results = []
for i, task in enumerate(test_tasks, 1):
    print(f"\n{'='*60}")
    print(f"Ejecutando tarea {i}/{len(test_tasks)}")
    print(f"{'='*60}")
    
    result = run_multi_agent_workflow(
        multi_agent_workflow,
        task,
        f"multi_agent_task_{i}"
    )
    multi_agent_results.append(result)

## 8. Análisis de Experimentos con MLflow

In [ ]:
from mlflow.tracking import MlflowClient

client = MlflowClient()
experiment = client.get_experiment_by_name(experiment_name)

# Obtener todos los runs
runs = client.search_runs(
    experiment_ids=[experiment.experiment_id],
    order_by=["start_time DESC"]
)

print(f"📊 Total de runs: {len(runs)}\n")

# Analizar métricas
runs_data = []
for run in runs:
    runs_data.append({
        'run_name': run.info.run_name,
        'workflow_type': run.data.params.get('workflow_type', 'unknown'),
        'task_type': run.data.params.get('task_type', 'N/A'),
        'selected_agent': run.data.params.get('selected_agent', 'N/A'),
        'execution_time': run.data.metrics.get('execution_time_seconds', 0),
        'llm_calls': run.data.metrics.get('llm_calls', 0),
        'total_messages': run.data.metrics.get('total_messages', 0)
    })

runs_df = pd.DataFrame(runs_data)
print("📊 Resumen de Experimentos:")
print(runs_df.to_string(index=False))

# Estadísticas por workflow type
print("\n📊 Estadísticas por Tipo de Workflow:")
stats = runs_df.groupby('workflow_type').agg({
    'execution_time': ['mean', 'min', 'max'],
    'llm_calls': ['mean', 'sum'],
    'run_name': 'count'
}).round(3)
print(stats)

# Análisis de agentes
multi_agent_runs = runs_df[runs_df['workflow_type'] == 'multi_agent']
if len(multi_agent_runs) > 0:
    print("\n📊 Uso de Agentes:")
    print(multi_agent_runs['selected_agent'].value_counts())

## 9. Visualizaciones

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configurar estilo
sns.set_style('whitegrid')

# Gráfico 1: Tiempo de ejecución por workflow type
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# Execution time
runs_df.boxplot(column='execution_time', by='workflow_type', ax=axes[0])
axes[0].set_title('Tiempo de Ejecución por Tipo de Workflow')
axes[0].set_xlabel('Tipo de Workflow')
axes[0].set_ylabel('Tiempo (segundos)')
plt.sca(axes[0])
plt.xticks(rotation=45)

# LLM calls
runs_df.boxplot(column='llm_calls', by='workflow_type', ax=axes[1])
axes[1].set_title('Llamadas LLM por Tipo de Workflow')
axes[1].set_xlabel('Tipo de Workflow')
axes[1].set_ylabel('Número de Llamadas')
plt.sca(axes[1])
plt.xticks(rotation=45)

plt.tight_layout()
plt.savefig('workflow_metrics.png', dpi=150, bbox_inches='tight')
plt.show()

# Gráfico 2: Distribución de agentes (si hay runs multi-agent)
if len(multi_agent_runs) > 0:
    plt.figure(figsize=(10, 6))
    agent_counts = multi_agent_runs['selected_agent'].value_counts()
    colors = sns.color_palette('viridis', len(agent_counts))
    agent_counts.plot(kind='bar', color=colors)
    plt.title('Distribución de Uso de Agentes', fontsize=14)
    plt.xlabel('Agente')
    plt.ylabel('Número de Veces Usado')
    plt.xticks(rotation=45)
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.savefig('agent_distribution.png', dpi=150, bbox_inches='tight')
    plt.show()

print("✅ Visualizaciones guardadas")

## 10. Comparación de Prompts (A/B Testing)

In [ ]:
def test_prompt_variations():
    """
    Compara diferentes variaciones de prompts
    """
    
    prompt_versions = {
        "v1_simple": "Resume el siguiente texto: {text}",
        "v2_detailed": "Por favor, proporciona un resumen conciso y detallado del siguiente texto, destacando los puntos clave: {text}",
        "v3_bullet": "Resume el siguiente texto en formato de bullet points: {text}"
    }
    
    test_text = "Machine Learning es una rama de la inteligencia artificial que permite a las computadoras aprender sin ser explícitamente programadas. Utiliza algoritmos para encontrar patrones en datos."
    
    for version_name, prompt_template in prompt_versions.items():
        with mlflow.start_run(run_name=f"prompt_test_{version_name}"):
            
            # Log prompt version
            mlflow.log_param("prompt_version", version_name)
            mlflow.log_param("prompt_template", prompt_template)
            mlflow.log_param("test_text", test_text)
            
            # Ejecutar
            prompt = prompt_template.format(text=test_text)
            llm.call_count = 0
            
            start_time = datetime.now()
            response = llm.invoke([HumanMessage(content=prompt)])
            end_time = datetime.now()
            
            execution_time = (end_time - start_time).total_seconds()
            
            # Log métricas
            mlflow.log_metric("execution_time_seconds", execution_time)
            mlflow.log_metric("response_length", len(response.content))
            mlflow.log_metric("llm_calls", llm.call_count)
            
            # Guardar respuesta
            with open(f'response_{version_name}.txt', 'w') as f:
                f.write(response.content)
            mlflow.log_artifact(f'response_{version_name}.txt')
            
            # Tags
            mlflow.set_tag("test_type", "prompt_comparison")
            mlflow.set_tag("prompt_version", version_name)
            
            print(f"\n{'='*60}")
            print(f"Prompt Version: {version_name}")
            print(f"{'='*60}")
            print(f"Prompt: {prompt}")
            print(f"\nRespuesta: {response.content}")
            print(f"Tiempo: {execution_time:.3f}s")

test_prompt_variations()

## 11. Best Practices y Recomendaciones

In [ ]:
print("💡 MEJORES PRÁCTICAS: MLFLOW + LANGGRAPH")
print("=" * 60)

print("\n1️⃣ TRACKING:")
print("   ✅ Loggea todos los parámetros del LLM (model, temperature, etc.)")
print("   ✅ Trackea tiempo de ejecución y número de llamadas")
print("   ✅ Guarda prompts y respuestas como artefactos")
print("   ✅ Usa tags para categorizar experimentos")

print("\n2️⃣ VERSIONADO:")
print("   ✅ Versiona tus prompts y guárdalos en MLflow")
print("   ✅ Usa el Model Registry para producción")
print("   ✅ Mantén historial de cambios en workflows")

print("\n3️⃣ EVALUACIÓN:")
print("   ✅ Define métricas claras (latencia, calidad, costo)")
print("   ✅ Compara diferentes modelos y configuraciones")
print("   ✅ A/B testing de prompts")

print("\n4️⃣ LANGGRAPH:")
print("   ✅ Diseña workflows con estados claros")
print("   ✅ Usa agentes especializados para tareas específicas")
print("   ✅ Implementa routing inteligente")
print("   ✅ Maneja errores y reintentos")

print("\n5️⃣ PRODUCCIÓN:")
print("   ✅ Monitorea costos de API")
print("   ✅ Implementa caching para respuestas comunes")
print("   ✅ Usa rate limiting")
print("   ✅ Loggea errores y excepciones")

print("\n6️⃣ SEGURIDAD:")
print("   ✅ NUNCA loggees API keys")
print("   ✅ Sanitiza inputs de usuario")
print("   ✅ Implementa validación de outputs")
print("   ✅ Usa variables de entorno para secrets")

print("\n" + "=" * 60)

## 12. Resumen y Próximos Pasos

In [ ]:
# Estadísticas finales
all_runs = client.search_runs(
    experiment_ids=[experiment.experiment_id]
)

print("🎉 RESUMEN DEL TUTORIAL")
print("=" * 60)

print(f"\n📊 Experimento: {experiment_name}")
print(f"📊 Total de runs: {len(all_runs)}")

# Contar por tipo
workflow_types = {}
for run in all_runs:
    wf_type = run.data.params.get('workflow_type', 'unknown')
    workflow_types[wf_type] = workflow_types.get(wf_type, 0) + 1

print(f"\n📊 Runs por tipo de workflow:")
for wf_type, count in workflow_types.items():
    print(f"   {wf_type}: {count}")

print(f"\n✅ Conceptos aprendidos:")
print(f"   - Integración MLflow + LangGraph")
print(f"   - Workflows simples y multi-agente")
print(f"   - Tracking de prompts y respuestas")
print(f"   - Comparación de configuraciones")
print(f"   - A/B testing de prompts")
print(f"   - Métricas y evaluación")

print(f"\n🚀 Próximos pasos:")
print(f"   - Integrar con LLMs reales (OpenAI, Anthropic, etc.)")
print(f"   - Implementar RAG (Retrieval-Augmented Generation)")
print(f"   - Agregar evaluación automática de calidad")
print(f"   - Deployar workflows en producción")
print(f"   - Implementar human-in-the-loop")
print(f"   - Monitoreo y alertas")

print(f"\n💻 Ver resultados: mlflow ui --port 5000")
print(f"\n📁 Artefactos guardados:")
print(f"   - Conversaciones JSON")
print(f"   - Resultados de agentes")
print(f"   - Visualizaciones")
print(f"   - Respuestas de prompts")

print("\n" + "=" * 60)
print("✅ Tutorial completado exitosamente!")